In [3]:
%matplotlib widget

from netCDF4 import Dataset
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage.measure import label, regionprops
import matplotlib.patches as mpatches
from datetime import *

#rootgrp = Dataset("raw/2015-04-01-H00-ddm.nc", "r", format="NETCDF4")
#metagrp = Dataset("raw/2015-04-01-H00-metadata.nc", "r", format="NETCDF4")
#group = '000095'
#index = 525;

file_root_name = 'raw/L1B/2018-07-29-H18'
rootgrp = Dataset(os.path.join(os.environ['TDS_ROOT'], file_root_name+'-DDMs.nc') , "r", format="NETCDF4")
metagrp = Dataset(os.path.join(os.environ['TDS_ROOT'], file_root_name+'-metadata.nc') , "r", format="NETCDF4")
group = '000050'
index = 386


def find_index_meta(group, index):
    datenum = rootgrp.groups[group].variables['IntegrationMidPointTime'][index]
    #print(datenum)

    datenums_meta = np.array(metagrp.variables['IntegrationMidPointTime'])
    #print(datenums_meta)

    for i, datenum_meta in enumerate(datenums_meta):
        if datenum_meta == datenum:
            #print("Foud: {0} | {1}".format(i, datenum_meta))
            return i
            break

index_meta = find_index_meta(group,index)
print("index meta: {}".format(index_meta))

def calculate_delay_increment_seconds(delay_pixel, group, index):
    delay_in_samples = delay_pixel * metagrp.groups[group].CodeDelaySpacingSamplesBetweenPixels
    delay_in_seconds = delay_in_samples / metagrp.groups[group].SamplingFrequency
    delay_of_tracking_point_in_seconds = (metagrp.groups[group].TrackingOffsetDelayNs - metagrp.groups[group].variables['SpecularPathRangeOffset'][index])*1e-9
    delay_increment_seconds = delay_in_seconds - delay_of_tracking_point_in_seconds
    return delay_increment_seconds

def calculate_delay_increment_chips(delay_pixel, group, index):
    chips_per_second = 1.023e6
    return calculate_delay_increment_seconds(delay_pixel, group, index)*chips_per_second

def calculate_doppler_increment(doppler_pixel, group, index):
    return doppler_pixel*metagrp.groups[group].DopplerResolution - metagrp.groups[group].TrackingOffsetDopplerHz

import matplotlib.pyplot as plt
%matplotlib widget
import numpy as np

x_col = list(map(lambda x: calculate_delay_increment_chips(x,group,index), np.arange(0,128)))
print(x_col[70])
y_col = list(map(lambda x: calculate_doppler_increment(x,group,index), np.arange(-10,10)))
x, y = np.meshgrid(x_col, y_col)

ddm = rootgrp.groups[group].variables['DDM'][index].data

datenum = rootgrp.groups[group].variables['IntegrationMidPointTime'][index]
datenum_meta = metagrp.variables['IntegrationMidPointTime'][index_meta]
print("datenum group: {0}\ndatenum meta: {1}".format(datenum, datenum_meta))
lat = metagrp.groups[group].variables['SpecularPointLat'][index]
lon = metagrp.groups[group].variables['SpecularPointLon'][index]

fig, ax = plt.subplots(1,figsize=(10, 4))
cs = ax.contourf(x, y, ddm, 30, cmap='viridis')
ax.set_title('DDM')
ax.grid(c='k', ls='-', alpha=0.3)

plt.show()

print(calculate_delay_increment_seconds(69,group,index))
print(calculate_doppler_increment(2,group,index))

index meta: 11912
1.5000926717113745
datenum group: 737270.7678935069
datenum meta: 737270.7678935069


FigureCanvasNbAgg()

1.2219720401499338e-06
1000.0


In [4]:
# https://en.wikipedia.org/wiki/IERS_Reference_Meridian: 
# It is also the reference meridian of the Global Positioning System (GPS) operated by the United States Department of Defense, and of WGS84 

w_earth = np.array([0, 0, 7.2921158553e-5]) # rad/sec

r_tx = metagrp.groups[group].variables['TransmitterPositionX'][index]
r_ty = metagrp.groups[group].variables['TransmitterPositionY'][index]
r_tz = metagrp.groups[group].variables['TransmitterPositionZ'][index]
r_t = np.array([r_tx,r_ty,r_tz])

v_tx = metagrp.groups[group].variables['TransmitterVelocityX'][index]
v_ty = metagrp.groups[group].variables['TransmitterVelocityY'][index]
v_tz = metagrp.groups[group].variables['TransmitterVelocityZ'][index]
v_t = np.array([v_tx,v_ty,v_tz])
# inertial velocity
v_t_i = v_t + np.cross(w_earth, r_t)

from datetime import *
from dateutil import tz

def datenum_to_pytime(matlab_datenum):
    python_datetime = datetime.fromordinal(int(matlab_datenum)) + timedelta(days=matlab_datenum%1) - timedelta(days = 366)
    return python_datetime.strftime('%Y-%m-%d %H:%M:%S')

datenum = rootgrp.groups[group].variables['IntegrationMidPointTime'][index]
print(datenum_to_pytime(float(datenum)))

datenum_meta = metagrp.variables['IntegrationMidPointTime'][index_meta]
print(datenum_to_pytime(float(datenum_meta)))

r_rx = metagrp.variables['ReceiverPositionX'][index_meta]
r_ry = metagrp.variables['ReceiverPositionY'][index_meta]
r_rz = metagrp.variables['ReceiverPositionZ'][index_meta]
r_r = np.array([r_rx,r_ry,r_rz])

v_rx = metagrp.variables['ReceiverVelocityX'][index_meta]
v_ry = metagrp.variables['ReceiverVelocityY'][index_meta]
v_rz = metagrp.variables['ReceiverVelocityZ'][index_meta]
v_r = np.array([v_rx, v_ry, v_rz])
# inertial velocity
v_r_i = v_r + np.cross(w_earth, r_r)

r_spx = metagrp.groups[group].variables['SpecularPointPositionX'][index]
r_spy = metagrp.groups[group].variables['SpecularPointPositionY'][index]
r_spz = metagrp.groups[group].variables['SpecularPointPositionZ'][index]
r_sp = np.array([r_spx,r_spy,r_spz])

i_sp = metagrp.groups[group].variables['SPIncidenceAngle'][index]
print(i_sp)

2018-07-29 18:25:45
2018-07-29 18:25:45
23.336813


In [6]:
from scipy.optimize import fsolve

r_sp = np.array([ 2948659.421067,  -3224178.67975644,  4631001.4712043 ])

def unit_vector(vector):
    """ Returns the unit vector of the vector. """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2' """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.dot(v1_u, v2_u))

def random_3d_vector():
    """
    Generates a random 3D unit vector (direction) with a uniform spherical distribution
    Algo from http://stackoverflow.com/questions/5408276/python-uniform-spherical-distribution
    :return:
    """
    phi = np.random.uniform(0,np.pi*2)
    costheta = np.random.uniform(-1,1)

    theta = np.arccos( costheta )
    x = np.sin( theta) * np.cos( phi )
    y = np.sin( theta) * np.sin( phi )
    z = np.cos( theta )
    return np.array([x,y,z])


R = np.linalg.norm(r_sp)

# Speed of light
c = 299792458 #m/s
# GPS L1 center frequency
f_c = 1575.42e6 # Hz 

def equations(p):
    x,y,z = p
    r = np.array([x,y,z])
    
    f = -2000 # Hz
    chips_per_second = 1.023e6
    delay = 5/chips_per_second

    tau = c*delay
    tau_r = np.linalg.norm(r-r_t) + np.linalg.norm(r_r-r)
    tau_sp = np.linalg.norm(r_sp-r_t) + np.linalg.norm(r_r-r_sp)
    f1 = -tau + (tau_r - tau_sp)
    
    f_d_r = f_c/c*(np.dot(v_t,unit_vector(r-r_t)) - np.dot(v_r,unit_vector(r_r-r)))
    f_d_sp = f_c/c*(np.dot(v_t,unit_vector(r_sp-r_t)) - np.dot(v_r,unit_vector(r_r-r_sp)))
    f2 = -f + (f_d_r - f_d_sp)
    
    f3 = R - np.linalg.norm(r)
    
    return (f1, f2, f3)

sol_1 = np.array(fsolve(equations, r_sp, xtol=1e-8, maxfev=1000, epsfcn=1e-24))
sol_2 = sol_1
k = 1
while np.linalg.norm(sol_1 - sol_2) < 0.5:
    r_0 =  r_sp + random_3d_vector()*1e3*k
    sol_2 = np.array(fsolve(equations,r_0))
    k = k+1
    
print("sol1: {}".format(sol_1))
print("sol2: {}".format(sol_2))

r_sol = np.array(sol_1)

print(r_sp)
print(r_sol)

lon_increment = (np.arctan2(r_sol[1],r_sol[0]) - np.arctan2(r_sp[1],r_sp[0]))*180/np.pi
lat_increment = (np.arctan2(r_sol[2],np.linalg.norm([r_sol[0],r_sol[1]])) - np.arctan2(r_sp[2],np.linalg.norm([r_sp[0],r_sp[1]])))*180/np.pi
print(lat_increment)
print(lon_increment)

lon_sp = np.arctan2(r_sp[1],r_sp[0])*180/np.pi
lat_sp = np.arcsin(abs(r_sp[2]/np.linalg.norm(r_sp)))*180/np.pi

lat = lat_sp + lat_increment
lon = lon_sp + lon_increment

lon = np.arctan2(r_sol[1],r_sol[0])*180/np.pi
lat = np.arcsin(abs(r_sol[2]/np.linalg.norm(r_sol)))*180/np.pi

print(lat_sp)
print(lon_sp)

print(lat)
print(lon)

print('-------------')

r_sol = np.array(sol_2)

print(r_sp)
print(r_sol)

lon_increment = (np.arctan2(r_sol[1],r_sol[0]) - np.arctan2(r_sp[1],r_sp[0]))*180/np.pi
lat_increment = (np.arctan2(r_sol[2],np.linalg.norm([r_sol[0],r_sol[1]])) - np.arctan2(r_sp[2],np.linalg.norm([r_sp[0],r_sp[1]])))*180/np.pi
print(lat_increment)
print(lon_increment)

lon_sp = np.arctan2(r_sp[1],r_sp[0])*180/np.pi
lat_sp = np.arcsin(abs(r_sp[2]/np.linalg.norm(r_sp)))*180/np.pi

lat = lat_sp + lat_increment
lon = lon_sp + lon_increment

lon = np.arctan2(r_sol[1],r_sol[0])*180/np.pi
lat = np.arcsin(abs(r_sol[2]/np.linalg.norm(r_sol)))*180/np.pi

print(lat_sp)
print(lon_sp)

print(lat)
print(lon)


sol1: [ 2929156.82402755 -3203852.11955866  4657409.90927228]
sol2: [ 2793536.45237771 -3274345.19771806  4691578.92208191]
[ 2948659.421067   -3224178.67975644  4631001.4712043 ]
[ 2929156.82402755 -3203852.11955866  4657409.90927228]
0.3474272065013944
-0.008892591742562217
46.66616387703584
-47.55565104846608
47.01359108353724
-47.56454364020865
-------------
[ 2948659.421067   -3224178.67975644  4631001.4712043 ]
[ 2793536.45237771 -3274345.19771806  4691578.92208191]
0.8003365911341148
-1.9749040471012416
46.66616387703584
-47.55565104846608
47.46650046816996
-49.53055509556733


In [ ]:
a = 6378137 #m
b = 6356752.314245 #m

F = np.array([[1/a,0,0],[0,1/a,0],[0,0,1/b]])
r_r_F = F@r_r
r_t_F = F@r_t
r_sp_F = F@r_sp

print(r_r_F)
print(r_t_F)
print(r_sp_F)
r_i = r_sp_F - r_t_F
r_s = r_r_F - r_sp_F
print(np.linalg.norm(r_sp_F))
print(angle_between(r_i, r_sp_F)*180/np.pi)
print(angle_between(r_s, r_sp_F)*180/np.pi)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def set_axes_equal(ax):
    '''Make axes of 3D plot have equal scale so that spheres appear as spheres,
    cubes as cubes, etc..  This is one possible solution to Matplotlib's
    ax.set_aspect('equal') and ax.axis('equal') not working for 3D.

    Input
      ax: a matplotlib axis, e.g., as output from plt.gca().
    '''

    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    # The plot bounding box is a sphere in the sense of the infinity
    # norm, hence I call half the max range the plot radius.
    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_aspect('equal')
r_sp2_F = F@np.array([res_x,res_y,res_z])
ax.scatter(r_sp2_F[0],r_sp2_F[1],r_sp2_F[2])
#ax.scatter(r_sp_F[0],r_sp_F[1],r_sp_F[2])
ax.scatter(r_t_F[0],r_t_F[1],r_t_F[2])
ax.scatter(r_r_F[0],r_r_F[1],r_r_F[2])

# Make data
u = np.linspace(0, 2*np.pi, 10)
v = np.linspace(0, np.pi, 10)
x = 0.7 * np.outer(np.cos(u), np.sin(v))
y = 0.7 * np.outer(np.sin(u), np.sin(v))
z = 0.7 * np.outer(np.ones(np.size(u)), np.cos(v))

# Plot the surface
ax.plot_surface(x, y, z, color='b')
set_axes_equal(ax)

print(angle_between(r_sp2_F, (r_r_F-r_sp2_F))*180/np.pi)
print(angle_between(r_sp2_F, (r_t_F-r_sp2_F))*180/np.pi)

In [ ]:
import numpy as np
from scipy.optimize import minimize

def fun(p):
    x,y,z = p
    r = np.array([x,y,z])
    # r_t is the transmitter position
    # r_r is the receiver position
    #tau_sp = np.linalg.norm(r-r_t) + np.linalg.norm(r_r-r)
    f = angle_between(r_norm, (r_r-r)) - angle_between(r_norm, (r_t-r))
    return tau_sp

def cons(p):
    a = 6378137 #m
    b = 6356752.314245 #m
    x,y,z = p
    return (x/a)**2 + (y/a)**2 + (z/b)**2 - 1
    
res = minimize(fun, r_sp, method='SLSQP', options={'ftol':1e-24,'eps':1e-16,'maxiter':1e8}, constraints={'type':'eq','fun': cons}, tol=1e-24)

res_x,res_y,res_z = res.x
r_sol = np.array([res_x,res_y,res_z])

lon_computed = np.arctan2(r_sol[1],r_sol[0])*180/np.pi
lat_computed = np.arcsin(abs(r_sol[2]/np.linalg.norm(r_sol)))*180/np.pi

lat_sp = metagrp.groups[group].variables['SpecularPointLat'][index].data
lon_sp = metagrp.groups[group].variables['SpecularPointLon'][index].data

print("TDS lat sp: {0} \nTDS lon sp: {1}\n".format(lat_sp, lon_sp) )

print("Computed lat sp: {0} \nComputed lon sp computd: {1}\n".format(lat_computed, lon_computed) )

print("TDS incidence angle {0:.2f} \nTDS reflection angle: {1:.2f}".format(\
    angle_between(r_sp, (r_r-r_sp))*180/np.pi,\
    angle_between(r_sp, (r_t-r_sp))*180/np.pi))

i_sp = metagrp.groups[group].variables['SPIncidenceAngle'][index]
print("TDS incidence angle from metadata: {0}\n".format(i_sp))

print("Computed incidence angle {0:.2f} \nComputed reflection angle: {1:.2f}\n".format(\
    angle_between(r_sol, (r_r-r_sol))*180/np.pi,\
    angle_between(r_sol, (r_t-r_sol))*180/np.pi))


In [ ]:
# Make data
u = np.linspace(0, 2*np.pi, 10)
v = np.linspace(0, np.pi, 10)
x = R * np.outer(np.cos(u), np.sin(v))
y = R * np.outer(np.sin(u), np.sin(v))
z = R * np.outer(np.ones(np.size(u)), np.cos(v))

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_aspect('equal')

min = R*1e5;
for i, x_i in enumerate(x):
    for j, x_j in enumerate(x_i):
        ax.scatter(x[i][j],y[i][j],z[i][j])
    

set_axes_equal(ax)

In [ ]:
def unit_vector(vector):
    """ Returns the unit vector of the vector. """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2' """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.dot(v1_u, v2_u))

# Create points on the WGS-84 ellipsoide
a = 6378137 #m
b = 6356752.314245 #m
u = np.linspace(0, 2*np.pi, 1000)
v = np.linspace(0, np.pi, 1000)
x = a * np.outer(np.cos(u), np.sin(v))
y = a * np.outer(np.sin(u), np.sin(v))
z = b * np.outer(np.ones(np.size(u)), np.cos(v))

# Compute the distnace to transmiter and reciever from points 
# on WGS-84 ellipsoid and find the minimum distance. 
min = np.inf;
r_min = np.zeros(3)
for i, x_i in enumerate(x):
    for j, x_j in enumerate(x_i):
        r = np.array([x[i][j],y[i][j],z[i][j]])
        d = np.linalg.norm(r-r_t) + np.linalg.norm(r_r-r)
        if d < min:
            min = d
            r_min = r

lon_computed = np.arctan2(r_min[1],r_min[0])*180/np.pi
lat_computed = np.arcsin(abs(r_min[2]/np.linalg.norm(r_min)))*180/np.pi

lat_sp = metagrp.groups[group].variables['SpecularPointLat'][index].data
lon_sp = metagrp.groups[group].variables['SpecularPointLon'][index].data

print("TDS lat sp: {0} \nTDS lon sp: {1}\n".format(lat_sp, lon_sp) )

print("Computed lat sp: {0} \nComputed lon sp computd: {1}\n".format(lat_computed, lon_computed) )

print("TDS incidence angle {0:.2f} \nTDS reflection angle: {1:.2f}".format(\
    angle_between(r_sp, (r_t-r_sp))*180/np.pi,\
    angle_between(r_sp, (r_r-r_sp))*180/np.pi))

i_sp = metagrp.groups[group].variables['SPIncidenceAngle'][index]
print("TDS incidence angle from metadata: {0}\n".format(i_sp))

print("Computed incidence angle {0:.2f} \nComputed reflection angle: {1:.2f}\n".format(\
    angle_between(r_min, (r_r-r_min))*180/np.pi,\
    angle_between(r_min, (r_t-r_min))*180/np.pi))

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def set_axes_equal(ax):
    '''Make axes of 3D plot have equal scale so that spheres appear as spheres,
    cubes as cubes, etc..  This is one possible solution to Matplotlib's
    ax.set_aspect('equal') and ax.axis('equal') not working for 3D.

    Input
      ax: a matplotlib axis, e.g., as output from plt.gca().
    '''

    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    # The plot bounding box is a sphere in the sense of the infinity
    # norm, hence I call half the max range the plot radius.
    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_aspect('equal')

# Make data
u = np.linspace(0, 2*np.pi, 10)
v = np.linspace(0, np.pi, 10)
x = 0.5* a * np.outer(np.cos(u), np.sin(v))
y = 0.5* a * np.outer(np.sin(u), np.sin(v))
z = 0.5* a * np.outer(np.ones(np.size(u)), np.cos(v))

ax.scatter(r_r[0],r_r[1],r_r[2])
ax.scatter(r_t[0],r_t[1],r_t[2])
ax.scatter(r_sol[0],r_sol[1],r_sol[2])
ax.scatter(r_sp[0],r_sp[1],r_sp[2])

# Plot the surface
ax.plot_surface(x, y, z, color='b')
set_axes_equal(ax)

In [14]:
a = 6378137 #m
b = 6356752.314245 #m

def ellip_norm(r):
    x,y,z = r
    return np.array([2*x/a**2,2*y/a**2,2*z/b**2])
    
r_center = np.array(r_sp)
for it in range(3):
    r_inc = 100e3/10**(it)
    r_step = 1e3/10**(it)

    x = np.arange(r_center[0]-r_inc, r_center[0]+r_inc, r_step)
    y = np.arange(r_center[1]-r_inc, r_center[1]+r_inc, r_step)
    xx, yy = np.meshgrid(x, y)
    z = b*(1-(xx/a)**2-(yy/a)**2)**(1/2)

    #fig = plt.figure()
    #ax = fig.add_subplot(111, projection='3d')
    #ax.set_aspect('equal')
    #ax.plot_surface(xx,yy,z)
    #set_axes_equal(ax)

    min = np.inf
    for x_i, x_val in enumerate(x):
        for y_i, y_val in enumerate(y):
            z_val = z[y_i,x_i]
            r = np.array([x_val, y_val, z_val])
            d = np.linalg.norm(r-r_t) + np.linalg.norm(r_r-r)
            if d < min:
                min = d
                r_min = r

    r_sol = r_min
    print(r_sol)

    lon_computed = np.arctan2(r_sol[1],r_sol[0])*180/np.pi
    lat_computed = np.arcsin(abs(r_sol[2]/np.linalg.norm(r_sol)))*180/np.pi

    lat_sp = metagrp.groups[group].variables['SpecularPointLat'][index].data
    lon_sp = metagrp.groups[group].variables['SpecularPointLon'][index].data

    print("TDS lat sp: {0} \nTDS lon sp: {1}\n".format(lat_sp, lon_sp) )

    print("Computed lat sp: {0} \nComputed lon sp computd: {1}\n".format(lat_computed, lon_computed) )

    print("TDS incidence angle {0:.2f} \nTDS reflection angle: {1:.2f}".format(\
        angle_between(ellip_norm(r_sp), (r_r-r_sp))*180/np.pi,\
        angle_between(ellip_norm(r_sp), (r_t-r_sp))*180/np.pi))

    print("Computed incidence angle {0:.2f} \nComputed reflection angle: {1:.2f}\n".format(\
        angle_between(ellip_norm(r_sol), (r_r-r_sol))*180/np.pi,\
        angle_between(ellip_norm(r_sol), (r_t-r_sol))*180/np.pi))
    
    r_center = r_sol


[ 2880659.421067   -3288178.67975644  4628813.45304513]
TDS lat sp: 46.858436584472656 
TDS lon sp: -48.786739349365234

Computed lat sp: 46.63737660131846 
Computed lon sp computd: -48.77952157140189

TDS incidence angle 27.58 
TDS reflection angle: 22.88
Computed incidence angle 23.32 
Computed reflection angle: 23.30

[ 2880259.421067   -3288678.67975644  4628707.86609264]
TDS lat sp: 46.858436584472656 
TDS lon sp: -48.786739349365234

Computed lat sp: 46.635987807992244 
Computed lon sp computd: -48.787783090424405

TDS incidence angle 27.58 
TDS reflection angle: 22.88
Computed incidence angle 23.30 
Computed reflection angle: 23.30

[ 2880269.421067   -3288628.67975644  4628736.97176556]
TDS lat sp: 46.858436584472656 
TDS lon sp: -48.786739349365234

Computed lat sp: 46.63637063355981 
Computed lon sp computd: -48.78725274029105

TDS incidence angle 27.58 
TDS reflection angle: 22.88
Computed incidence angle 23.30 
Computed reflection angle: 23.30

